<a href="https://colab.research.google.com/github/Beytab/RS6/blob/master/netflix_prize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip3 install surprise

In [5]:
!pwd
!ls -R

/content
.:
drive  sample_data

./drive:
'My Drive'

'./drive/My Drive':
'Colab Notebooks'

'./drive/My Drive/Colab Notebooks':
netflix-prize-data  netflix_prize.ipynb  test.ipynb

'./drive/My Drive/Colab Notebooks/netflix-prize-data':
combined_data_1.txt  combined_data_3.txt  movie_titles.csv  qualifying.txt
combined_data_2.txt  combined_data_4.txt  probe.txt	    README

./sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [9]:
#加载数据
import pandas as pd
df = pd.read_csv('./drive/My Drive/Colab Notebooks/netflix-prize-data/combined_data_2.txt',header=None,names=['uid','rating'],usecols=[0,1])
df['rating'] = df['rating'].astype(float)

#利用numpy数组将movie_id提取添加到dataframe中，此数据中movie_id是连续自然数，即从1到n
df_nan = pd.DataFrame(pd.isnull(df.rating))
print(df_nan)
df_nan = df_nan[df_nan['rating']==True]
df_nan = df_nan.reset_index()
print(df_nan)

          rating
0           True
1          False
2          False
3          False
4          False
...          ...
26982297   False
26982298   False
26982299   False
26982300   False
26982301   False

[26982302 rows x 1 columns]
         index  rating
0            0    True
1          259    True
2          855    True
3          960    True
4         2740    True
...        ...     ...
4706  26957496    True
4707  26959947    True
4708  26961403    True
4709  26980373    True
4710  26980497    True

[4711 rows x 2 columns]


In [10]:
data1.iloc[0][0][:-1]

'4500'

In [0]:
#加载数据
import pandas as pd
import numpy as np
def data_load(file_path):
  #读取数据
  df = pd.read_csv(file_path,header=None,names=['uid','rating'],usecols=[0,1])
  df['rating'] = df['rating'].astype(float)

  #利用numpy数组将movie_id提取添加到dataframe中，此数据中movie_id是连续自然数，即从1到n
  df_nan = pd.DataFrame(pd.isnull(df.rating))
  df_nan = df_nan[df_nan['rating']==True]
  df_nan = df_nan.reset_index()

  movie_np = []
  movie_id = int(df.iloc[0][0][:-1]) #获取第一部电影的id
  for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    temp = np.full((1,i-j-1),movie_id)
    movie_np = np.append(movie_np,temp)
    movie_id += 1
  last_record = np.full((1,len(df)-df_nan.iloc[-1,0]-1),movie_id)
  movie_np = np.append(movie_np,last_record)
  
  df = df[pd.notnull(df['rating'])]
  df['movie_id'] = movie_np.astype(int)
  df['uid'] = df['uid'].astype(int)
  return df
  
#进行数据筛选：筛选掉冷门电影和不活跃用户
def data_slicing(df):
  f = ['count','mean']
  df_movie_summary = df.groupby(by='movie_id')['rating'].agg(f)
  df_movie_summary.index = df_movie_summary.index.map(int)
  movie_benchmark = round(df_movie_summary['count'].quantile(0.95),0)
  drop_movie_list = df_movie_summary[df_movie_summary['count']<movie_benchmark].index
  
  df_user_summary = df.groupby(by='uid')['rating'].agg(f)
  df_user_summary.index = df_user_summary.index.map(int)
  user_benchmark = round(df_user_summary['count'].quantile(0.95),0)
  drop_user_list = df_user_summary[df_user_summary['count']<user_benchmark].index

  df = df[~df['movie_id'].isin(drop_movie_list)]
  df = df[~df['uid'].isin(drop_user_list)]
  return df

In [3]:
df1 = data_load('./drive/My Drive/Colab Notebooks/netflix-prize-data/combined_data_1.txt')
print(df1.shape)
df1 = data_slicing(df1)
print(df1.shape)
df2 = data_load('./drive/My Drive/Colab Notebooks/netflix-prize-data/combined_data_2.txt')
df2 = data_slicing(df2)
df3 = data_load('./drive/My Drive/Colab Notebooks/netflix-prize-data/combined_data_3.txt')
df3 = data_slicing(df3)
df4 = data_load('./drive/My Drive/Colab Notebooks/netflix-prize-data/combined_data_4.txt')
df4 = data_slicing(df4)
df = pd.concat([df1,df2,df3])
df.append(df4)
df = df.reset_index()
print('df size:',df.shape)
df.head()

(24053764, 3)
(2941349, 3)
df size: (8994033, 4)


,index,uid,rating,movie_id
0,52553,1990901,5.0,28
1,52556,765331,3.0,28
2,52557,1987434,4.0,28
3,52560,2193455,4.0,28
4,52561,1468812,4.0,28


In [5]:
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly,KNNBasic,NormalPredictor
from surprise import accuracy
from surprise.model_selection import KFold

reader = Reader(line_format='user item rating',sep=',',skip_lines=1)
data = Dataset.load_from_df(df[['uid','movie_id','rating']],reader=reader)
train_set = data.build_full_trainset()
# ALS优化
bsl_options = {'method':'als','n_epochs':5,'reg_u':12,'reg_i':5}
# SGD优化
#bsl_options = {'method':'sgd','n_epochs':5}
algo = BaselineOnly(bsl_options=bsl_options)
#algo = NormalPredictor()

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=4)
for trainset,testset in kf.split(data):
  algo.fit(trainset)
  predictions = algo.test(testset)
  # 计算RMSE
  accuracy.rmse(predictions,verbose=True)

Estimating biases using als...
RMSE: 0.8974
Estimating biases using als...
RMSE: 0.8971
Estimating biases using als...
RMSE: 0.8961
Estimating biases using als...
RMSE: 0.8972
